# 1) Decision Tree Implementation

In [1]:
import numpy as np
import pandas as pd

## a) Completing gini_index method.

In [2]:
#Create Split
## Split a dataset based on an attribute and an attribute value
def test_split(index, value, dataset):
    left, right = list(), list()
    for row in dataset:
        if row[index] < value:
            left.append(row)
        else:
            right.append(row)
    return left, right

## Calculate the Gini index for a split dataset
def gini_index(groups, classes):
    ## count all samples at split point.
    n_instances = float(sum([len(group) for group in groups]))
    ## sum weighted Gini index for each group
    gini = 0.0
    for group in groups:
        size = float(len(group))
        if size == 0:
            continue
        score = 0.0
        ## score (propability) the group based on the score for each class
        for class_val in classes:
            p = [row[-1] for row in group].count(class_val) / size
            score += p * p
    ## weight the group score by its relative size
        gini += (1.0 - score) * (size / n_instances)
    return gini
## Select the best split point for a dataset
def get_split(dataset):
    class_values = list(set(row[-1] for row in dataset))
    b_index, b_value, b_score, b_groups = 999, 999, 999, None
    for index in range(len(dataset[0])-1):
        for row in dataset:
            groups = test_split(index, row[index], dataset)
            gini = gini_index(groups, class_values)
            if gini < b_score:
                b_index, b_value, b_score, b_groups = index, row[index], gini, groups
    return {'index':b_index, 'value':b_value, 'groups':b_groups}

In [3]:
# Create a terminal node value
def to_terminal(group):
    outcomes = [row[-1] for row in group]
    return max(set(outcomes), key=outcomes.count)

# Create child splits for a node or make terminal
def split(node, max_depth, min_size, depth):
    left, right = node['groups']
    del(node['groups'])
    # check for a no split
    if not left or not right:
        node['left'] = node['right'] = to_terminal(left + right)
        return
    # check for max depth
    if depth >= max_depth:
        node['left'], node['right'] = to_terminal(left), to_terminal(right)
        return
    # process left child
    if len(left) <= min_size:
        node['left'] = to_terminal(left)
    else:
        node['left'] = get_split(left)
        split(node['left'], max_depth, min_size, depth+1)
    # process right child
    if len(right) <= min_size:
        node['right'] = to_terminal(right)
    else:
        node['right'] = get_split(right)
        split(node['right'], max_depth, min_size, depth+1)
          
# Build a decision tree
def build_tree(train, max_depth, min_size):
    root = get_split(train)
    split(root, max_depth, min_size, 1)
    return root

# Make a prediction with a decision tree
def predict(node, row):
    if row[node['index']] < node['value']:
        if isinstance(node['left'], dict):
            return predict(node['left'], row)
        else:
            return node['left']
    else:
        if isinstance(node['right'], dict):
            return predict(node['right'], row)
        else:
            return node['right']

## b) Uploading data.

In [4]:
filename = 'auto-mpg.data'
column_names = ['mpg', 'cylinders', 'displacement', 'horsepower', 'weight', 'acceleration', 'year', 'origin', 'name']
df = pd.read_csv(filename, delim_whitespace=True, names=column_names)

In [5]:
(df == '?').any()

mpg             False
cylinders       False
displacement    False
horsepower       True
weight          False
acceleration    False
year            False
origin          False
name            False
dtype: bool

In [6]:
df = df[df['horsepower'] != '?']

In [7]:
df

,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin,name
0,18.0,8,307.0,130.0,3504.0,12.0,70,1,chevrolet chevelle malibu
1,15.0,8,350.0,165.0,3693.0,11.5,70,1,buick skylark 320
2,18.0,8,318.0,150.0,3436.0,11.0,70,1,plymouth satellite
3,16.0,8,304.0,150.0,3433.0,12.0,70,1,amc rebel sst
4,17.0,8,302.0,140.0,3449.0,10.5,70,1,ford torino
...,...,...,...,...,...,...,...,...,...
393,27.0,4,140.0,86.00,2790.0,15.6,82,1,ford mustang gl
394,44.0,4,97.0,52.00,2130.0,24.6,82,2,vw pickup
395,32.0,4,135.0,84.00,2295.0,11.6,82,1,dodge rampage
396,28.0,4,120.0,79.00,2625.0,18.6,82,1,ford ranger


In [8]:
target_column = df.iloc[:, 0].copy()
df = df.drop(df.columns[0], axis=1)  
df = pd.concat([df, target_column],axis=1)
df = df.drop('name', axis=1)
df

,cylinders,displacement,horsepower,weight,acceleration,year,origin,mpg
0,8,307.0,130.0,3504.0,12.0,70,1,18.0
1,8,350.0,165.0,3693.0,11.5,70,1,15.0
2,8,318.0,150.0,3436.0,11.0,70,1,18.0
3,8,304.0,150.0,3433.0,12.0,70,1,16.0
4,8,302.0,140.0,3449.0,10.5,70,1,17.0
...,...,...,...,...,...,...,...,...
393,4,140.0,86.00,2790.0,15.6,82,1,27.0
394,4,97.0,52.00,2130.0,24.6,82,2,44.0
395,4,135.0,84.00,2295.0,11.6,82,1,32.0
396,4,120.0,79.00,2625.0,18.6,82,1,28.0


## c) Making mpg a binary target variable.

In [9]:
mu = df['mpg'].mean()

df['mpg'] = (df['mpg'] > mu).astype(int)

In [10]:
df

,cylinders,displacement,horsepower,weight,acceleration,year,origin,mpg
0,8,307.0,130.0,3504.0,12.0,70,1,0
1,8,350.0,165.0,3693.0,11.5,70,1,0
2,8,318.0,150.0,3436.0,11.0,70,1,0
3,8,304.0,150.0,3433.0,12.0,70,1,0
4,8,302.0,140.0,3449.0,10.5,70,1,0
...,...,...,...,...,...,...,...,...
393,4,140.0,86.00,2790.0,15.6,82,1,1
394,4,97.0,52.00,2130.0,24.6,82,2,1
395,4,135.0,84.00,2295.0,11.6,82,1,1
396,4,120.0,79.00,2625.0,18.6,82,1,1


## d) Growing tree with depth 1.

In [11]:
dataset = df.values

In [12]:
dataset[:,:-1].shape

(392, 7)

In [13]:
build_tree(dataset, 1, 1)

{'index': 1, 'value': 198.0, 'left': 1, 'right': 0}

In [14]:
df.columns[1]

'displacement'

## e) Verifying with scikit learn.

In [15]:
from sklearn.tree import DecisionTreeClassifier, export_text

X = df[df.columns[:-1]]  
y = df[df.columns[-1]]   

tree_classifier = DecisionTreeClassifier(max_depth=1, random_state=1)

tree_classifier.fit(X, y)

tree_rules = export_text(tree_classifier, feature_names=list(df.columns[:-1]))
print("Splitting Criteria:")
print(tree_rules)


Splitting Criteria:
|--- displacement <= 190.50
|   |--- class: 1
|--- displacement >  190.50
|   |--- class: 0



Both our code and sklearn code used displacement as the feature for splitting, where left child is class 1 and right child is class 0.

# 2) Random Forest Implementation

## a) Implementing Random Forest Method

In [126]:
from sklearn.tree import DecisionTreeClassifier
X = df[df.columns[:-1]]
y = df[df.columns[-1]]

def RandomForest(X, y, m, iterations):
    pred_dict = {}
    value_dict = {}
    for _ in range(iterations):
        selected_features = np.random.choice(X.columns, m, replace=True)
        # Replace = True since m in max number and not a set number.
        selected_features = list(set(selected_features))
        samples = X[selected_features]
        tree = DecisionTreeClassifier(max_depth=1, random_state=1)
        tree.fit(samples, y)
        first_split = selected_features[tree.tree_.feature[0]]
        first_split_value = tree.tree_.threshold[0]
        if first_split in pred_dict:
            pred_dict[first_split] += 1
            value_dict[first_split] = first_split_value
        else:
            pred_dict[first_split] = 1
            value_dict[first_split] = first_split_value
    
    predicted_class = max(pred_dict, key=lambda k: pred_dict[k])
    rates = {key: value / iterations for key, value in pred_dict.items()}

    return predicted_class, rates, value_dict

## b) Testing various m values

In [127]:
iters = [25, 50, 100, 125, 150]
m = [1,2,3,4,5,6]

In [128]:
scores = 0
for i in m:
    for j in iters:
        _, score, val = RandomForest(X, y, i, j)
        if score['displacement'] > scores:
            scores = score['displacement']
            best_params = (i, j)
print(f'Best params are: m = {best_params[0]} and Iterations = {best_params[1]} with score: {scores}')

Best params are: m = 6 and Iterations = 50 with score: 0.72


Every run gives different hyperparameters, but mostly, m= 6 and iterations = 50.

In [131]:
RandomForest(X, y, 6, 50)

('displacement',
 {'displacement': 0.72,
  'cylinders': 0.18,
  'origin': 0.02,
  'horsepower': 0.02,
  'weight': 0.06},
 {'displacement': 190.5,
  'cylinders': 5.5,
  'origin': 1.5,
  'horsepower': 93.5,
  'weight': 2764.5})

In [142]:
for i in m:
    rm, _, __ = RandomForest(X, y, i, 50)
    print(f'Highest selected feature for m as {i}: {rm} with rate = {_[rm]}')

Highest selected feature for m as 1: horsepower with rate = 0.24
Highest selected feature for m as 2: displacement with rate = 0.3
Highest selected feature for m as 3: displacement with rate = 0.48
Highest selected feature for m as 4: displacement with rate = 0.46
Highest selected feature for m as 5: displacement with rate = 0.52
Highest selected feature for m as 6: displacement with rate = 0.66


Displacement is always the chosen splitting criteria same as question 1d after we do majority voting provided enough iters and good m values are used.  

Also, when displacement is chosen, then value is same as 1d too since we use same classifier.  
  
Even in this case, many times features like cylinder are selected since during these runs, randomly selected features do no include displacement.  

If less number of iters and m is used this can give wrong results.  

For Example:

In [105]:
RandomForest(X, y, 2, 5)

('horsepower',
 {'horsepower': 0.4, 'acceleration': 0.2, 'year': 0.2, 'weight': 0.2},
 {'horsepower': 93.5, 'acceleration': 13.75, 'year': 79.5, 'weight': 2764.5})

We can see that displacement was not chosen even once.